# Creating and filling a consolidated table of M-ligue results

### 1. Setup

Define base folders, filenames, and URLs.
Only change the tournament_name and url each time you process a new tournament.

In [1]:
from pathlib import Path
import pandas as pd
import urllib.request
from openpyxl import load_workbook

# Base folder where game results are stored
base_dir = Path(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Tournaments\Game results")

# Consolidated results file
consolidated_path = base_dir / "Consolidated.xlsx"

# Define current tournament
tournament_name = "2025 July Final"

# Path for the downloaded tournament file
destination = base_dir / f"{tournament_name}.xlsx"

# Download URL for the tournament results
url = "https://chgk-world.ru/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6MzY0NTQyLCJwdXIiOiJibG9iX2lkIn19--a8a125f0ce9a166f4b2538e4858372fb2f36e56a/tournament-2034-table.xlsx"

### 2. Create Consolidated table for all tournaments

Run this step only the first time you set up the consolidated file.
It creates an empty Excel file with all required columns.

In [2]:
if not consolidated_path.exists():
    columns = ['Rank', 'Team', 'City', 'Team ID', 'Captain', 'Tour', 'Location', 'Status',
               'Total', 'Time', 'Question', 'Result', 'Tournament']
    pd.DataFrame(columns=columns).to_excel(consolidated_path, index=False)
    print("Created new Consolidated.xlsx file.")
else:
    print("Consolidated.xlsx already exists, skipping creation.")

Created new Consolidated.xlsx file.


### 3. Prepare Consolidated table

Start here if the file already exists.

In [3]:
df_consolidated = pd.read_excel(consolidated_path)
print(f"Loaded Consolidated file with {len(df_consolidated)} rows.")

Loaded Consolidated file with 0 rows.


### 4. Download the results of the particular game

After a game finishes, you can download its Excel results directly from chgk-world.ru.

In [4]:
import urllib.request
import ssl, certifi

# Fix SSL certificate verification (for urllib on Windows)
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

urllib.request.urlretrieve(url, destination)
print(f"Downloaded results to {destination}")

Downloaded results to C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Tournaments\Game results\2025 July Final.xlsx


### 5. Change '+' and '-' to '1' and '0'

In the downloaded results, correct and incorrect answers are shown as + and –.
We’ll convert them to numeric values (1 for correct, 0 for incorrect) for easier processing later.

In [5]:
replacement_pair = {"+": 1, "-": 0, "?": 0, None: 0, "": 0}

wb = load_workbook(destination)
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if cell.value in replacement_pair:
                cell.value = replacement_pair[cell.value]
wb.save(destination)

print("Replaced symbols with numeric values.")

Replaced symbols with numeric values.


### 6. Reshaping the table

Each question’s result is stored in a separate column.
We’ll convert those columns into rows so that each question gets its own record.

In [6]:
df_game = pd.read_excel(destination)

id_vars = [
    "#", "Название", "Город", "MakId", "Капитан", "тур 1", 
    "Площадка", "Зачет", "Сумма", "Время+"
]

df_game = df_game.melt(id_vars=id_vars, var_name="Question", value_name="Result")
df_game.to_excel(destination, index=False)
print("Transformed questions from columns to rows.")

Transformed questions from columns to rows.


### 7. Rename columns

Rename Russian headers to English ones for consistency in later visualization.

In [7]:
df_game.rename(columns={
    "#": "Rank",
    "Название": "Team",
    "Город": "City",
    "MakId": "Team ID",
    "Капитан": "Captain",
    "тур 1": "Tour",
    "Площадка": "Location",
    "Зачет": "Status",
    "Сумма": "Total",
    "Время+": "Time"
}, inplace=True)

print("Renamed columns.")

Renamed columns.


### 8. Add Tournament Name

Add a column to keep track of which tournament each record belongs to.

In [8]:
df_game["Tournament"] = tournament_name
df_game.to_excel(destination, index=False)

print(f"Added tournament name: {tournament_name}")

Added tournament name: 2025 July Final


### 9. Append Tournament Results to the Consolidated File

Finally, merge the cleaned tournament data into your consolidated table and save it.

In [9]:
df_updated = pd.concat([df_consolidated, df_game], ignore_index=True)
df_updated.to_excel(consolidated_path, index=False)

print(f"Appended {len(df_game)} new rows to Consolidated.xlsx.")

Appended 1980 new rows to Consolidated.xlsx.
